# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : Reproducibility
## Chapter : Curvature penalized models

In this notebook, we run some TTI models (Tilted Transversely Anisotropic), a kind of Finslerian manifold appearing in seismology, on the CPU and GPU, and check that they produce consistent results.

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Rander_Repro','GPU'))

In [2]:
import cupy as cp
import numpy as np
import itertools
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [3]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import FiniteDifferences as fd
from agd import LinearParallel as lp
from agd.Metrics.Seismic import Reduced
import agd.AutomaticDifferentiation.cupy_generic as cugen

norm_infinity = ad.Optimization.norm_infinity
from Notebooks_GPU.ExportedCode.Isotropic_Repro import RunCompare

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global HFMUtils,ad,cugen,Metrics,Reduced
    HFMUtils,ad,cugen,Metrics,Reduced = rreload([HFMUtils,ad,cugen,Metrics,Reduced],"../..")    
    HFMUtils.dictIn.RunSmart = cugen.cupy_get_args(HFMUtils.RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

In [5]:
cp = ad.functional.decorate_module_functions(cp,cugen.set_output_dtype32) # Use float32 and int32 types in place of float64 and int64
plt = ad.functional.decorate_module_functions(plt,cugen.cupy_get_args)
HFMUtils.dictIn.RunSmart = cugen.cupy_get_args(HFMUtils.RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

# 1. Defining a seismic norm

In [267]:
ReloadPackages()

In [16]:
xp=cp
n=2000
hfmIn_Constant = HFMUtils.dictIn({
    'model':'TTI2',
    'arrayOrdering':'RowMajor',
    'exportValues':1,
    'seeds':xp.array([[0.,0.]]),
    'factoringMethod':'Static',
    'factoringRadius':20,
#    'seedRadius':0,
    'order':2
#    'tips':[[x,y] for y in HFMUtils.CenteredLinspace(-1,1,6) 
#                    for x in HFMUtils.CenteredLinspace(-1,1,6)],
#    'exportGeodesicFlow':1,
})

hfmIn_Constant.SetRect(sides=[[-1,1],[-1,1]],dimx=n+1,sampleBoundary=True) # Define the domain
X = hfmIn_Constant.Grid() # Horizontal and vertical axis

lin = xp.array([1.,2.])
#quad = xp.zeros((2,2))
quad = xp.array([[0.,0.2],[0.2,0.]])
#quad = xp.array([[0.5,0.1],[0.1,-0.2]])
metric = Reduced(lin,quad).rotate_by(xp.array(0.5)) #Linear and quadratic part
hfmIn_Constant['metric'] = metric

Casting output of function array from float64 to float32
Casting output of function array from float64 to float32
Casting output of function array from float64 to float32
Casting output of function array from float64 to float32


In [17]:
metric.cost_bound()

array(    2, dtype=float32)

In [18]:
plt.axis('equal')
plt.contour(*X,metric.norm(X))

In [11]:
hfmOut = hfmIn_Constant.RunGPU()

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 14.31849479675293 seconds,  and 355 iterations.
Post-Processing
!! Warning !! Unused keys from user : ['factoringMethod']


In [36]:
plt.axis('equal')
plt.contour(*X,hfmOut['values'])

In [290]:
np.max(hfmOut['values']),np.max(metric.norm(X))

(array( 1.61, dtype=float32), array( 1.61, dtype=float32))

In [19]:
phi = (np.sqrt(5)-1)/2
psi = 1-phi
delta = psi*psi/(phi-psi)
phi,psi,delta

(0.6180339887498949, 0.3819660112501051, 0.6180339887498943)

In [19]:
hfmIn_Constant.RunSmart()

Field verbosity defaults to 1
Field seedRadius defaults to 2
Fast marching solver completed in 58.478 s.


{'FMCPUTime': 58.478,
 'GeodesicCPUTime': 0.0,
 'MaxStencilWidth': 2.0,
 'StencilCPUTime': 4.813,
 'defaulted': 'exportActiveNeighs exportActiveOffsets exportFactoringCenters exportFactoringRegion exportGeodesicFlow factoringPointChoice refineStencilAtWallBoundary seedRadius showProgress verbosity',
 'nAccepted': 4004001.0,
 'spreadedSeedGradients': array([[-0.804, -0.578],
        [0.932, 0.255],
        [0.804, 0.578],
        [0.932, 0.255],
        [0.611, -0.416],
        [ 0.24, 0.767],
        [ 0.24, 0.767],
        [-0.24, -0.767],
        [-0.611, 0.416],
        [-0.932, -0.255],
        [-0.804, -0.578]]),
 'spreadedSeedValues': array([1.57e-16, 0.00186, 0.00138, 0.000932, 0.00103, 0.00153, 0.000767, 0.000767, 0.00103, 0.000932, 0.00138]),
 'spreadedSeeds': array([[    0,     0],
        [-0.002,     0],
        [-0.001, -0.001],
        [-0.001,     0],
        [-0.001, 0.001],
        [    0, -0.002],
        [    0, -0.001],
        [    0, 0.001],
        [0.001, -0.001